# GEN DATA

In [1]:
import pandas as pd
from sklearn.datasets import make_blobs


make_blobs()

(array([[ 9.51361638,  8.40266751],
        [-4.1135632 , -4.06361557],
        [ 8.40450865,  5.1815009 ],
        [-3.62164286, -6.62262109],
        [-4.14766698, -7.17048979],
        [-2.89103135, -7.68782698],
        [-3.24689503, -7.05170298],
        [-3.6615665 , -5.94214027],
        [-1.78842185, -6.41072464],
        [-3.74306295, -6.73877505],
        [-4.39927389, -3.43030998],
        [ 7.40916453,  6.02097964],
        [ 8.3294002 ,  6.34912113],
        [-4.58395059, -5.88216665],
        [-4.67271194, -4.22172351],
        [-3.35702942, -7.82355856],
        [-2.58166324, -7.28670895],
        [-3.16959102, -5.01800561],
        [ 7.83491146,  7.07564596],
        [-1.39011182, -7.68498193],
        [-3.03326127, -4.63496965],
        [-4.65777876, -4.48097931],
        [ 7.63886595,  6.78787153],
        [-3.34978794, -4.76956396],
        [-7.00243893, -5.29669061],
        [ 8.36519833,  7.39361262],
        [-2.17375686, -6.04952143],
        [-4.98437693, -4.883

# GEN CENTERS

In [2]:
n = 3 # 3 dimensions
n_points = 2**n # max number of points before 2 points are proportional


In [3]:
# 3 DIMENSIONS
# COUNT IN BINARY TO GET COORDINATES

# 000
# 001
# 010
# 011
# 100
# 101
# 110
# 111

In [4]:
4
bin(4)
[eval(digit) for digit in  [*bin(4)[2:]]]

[1, 0, 0]

In [5]:
def clust_centers(n_dim, n_centers):
    # unary grid
    unary_grid = []
    for i in range(2**n_dim):
        unary_grid.append(
            [eval(digit) for digit in  [*format(i, f'#0{n_dim+2}b')[2:]]] # Get Binary representation and convert to list
            )
    if n_centers <= 2**n_dim:
        return unary_grid[:n_centers]
    grid = unary_grid
    double_grid = [(pd.Series(elem) * 2).tolist() for elem in unary_grid[1:]] # Double non null vectors
    grid.extend(double_grid[:n_centers-2**n_dim])
    return grid
clust_centers(2,7)

[[0, 0], [0, 1], [1, 0], [1, 1], [0, 2], [2, 0], [2, 2]]

In [6]:
import gen_data
df = gen_data.generate_data(n_clusters=5,clust_std=30*0.1, n_num=20,n_cat=10,n_indiv=250)
#df.columns = ['X','Y','Z','S','W']
#df.dtypes

In [7]:
df = gen_data.generate_data(n_clusters=9,clust_std=2,n_num=15,n_cat=15,cat_unique=3,n_indiv=250)

In [8]:
import plotly.express as px

In [9]:
from prince import FAMD
pca = FAMD(n_components=3)
rr = pca.fit_transform(df)
rr.columns = ['X','Y','Z']

px.scatter_3d(rr,'X','Y','Z')

In [10]:
from gen_data import generate_data
from prince import FAMD
import plotly.express as px

In [11]:
df = generate_data(n_clusters=5,clust_std=0.08,n_num=15,n_cat=15,cat_unique=3,n_indiv=250)
pca = FAMD(n_components=3)
rr = pca.fit_transform(df)
rr.columns = ['X','Y','Z']

px.scatter_3d(rr,'X','Y','Z')

In [12]:
import umap.umap_ as umap
n_components=3
intersection=False
df2 = df.copy()
numerical = df2.select_dtypes(exclude='object')
for c in numerical.columns:
    numerical[c] = (numerical[c] - numerical[c].mean())/numerical[c].std(ddof=0)
    
##preprocessing categorical
categorical = df2.select_dtypes(include='object')
categorical = pd.get_dummies(categorical)
#Embedding numerical & categorical
fit1 = umap.UMAP(#random_state=12,
                #n_neighbors=2,
                n_components=n_components).fit(numerical)
fit2 = umap.UMAP(metric='dice', 
                #n_neighbors=250,
                n_components=n_components).fit(categorical)
# intersection will resemble the numerical embedding more.
if intersection:
    embedding = fit1 * fit2

# union will resemble the categorical embedding more.
else:

    embedding = fit1 + fit2


um = pd.DataFrame(embedding.embedding_) # Each points' UMAP coordinate 

# Actual Plotting
um.columns = ['X','Y','Z']
#um['cluster'] = df['cluster'].astype(str)
fig = px.scatter_3d(um, 
                x='X',y='Y',z='Z')
fig.update_layout(showlegend=False)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

c:\Users\cleme\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

c:\Users\cleme\AppData\Local\Programs\Python\Python310\lib\site-packages\umap\umap_.py:1802: UserWarning:

gradient function is not yet implemented for dice distance metric; inverse_transform will be unavailable



In [13]:
import pacmap
pm = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)


In [15]:
import numpy as np
df2 = df.copy()
numerical = df2.select_dtypes(exclude='object')
for c in numerical.columns:
    numerical[c] = (numerical[c] - numerical[c].mean())/numerical[c].std(ddof=0)
    
##preprocessing categorical
categorical = df2.select_dtypes(include='object')
categorical = pd.get_dummies(categorical)
#Embedding numerical & categorical
fit1 = pacmap.PaCMAP(#random_state=12,
                n_components=n_components).fit_transform(numerical)

fit2 = pacmap.PaCMAP(distance='hamming', 
                n_components=n_components).fit_transform(categorical)
gamma = np.mean(np.std(numerical))/2
embedding = fit1+fit2*gamma

um = pd.DataFrame(embedding) # Each points' UMAP coordinate 

# Actual Plotting
um.columns = ['X','Y','Z']
#um['cluster'] = df['cluster'].astype(str)
fig = px.scatter_3d(um, 
                x='X',y='Y',z='Z')
fig.update_layout(showlegend=False)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

c:\Users\cleme\AppData\Local\Programs\Python\Python310\lib\site-packages\pacmap\pacmap.py:835: UserWarning:

apply_pca = True for Hamming distance. This option will be ignored.

